In [31]:
import tensorflow as tf
import scipy as sp
import numpy as np
import pandas as pd
import re
import hashlib
from sklearn.preprocessing import StandardScaler

import os
from pathlib import Path
import IPython.display as ipd
import cProfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
%matplotlib inline


from tensorflow.python.keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.utils import to_categorical

from sklearn.preprocessing import OrdinalEncoder

from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras.layers import Input, Dense, LSTM

from tensorflow.python.keras.layers import Embedding, Dropout, Activation

import sys
sys.path.insert(0, '../src/data')
import get_train_test

In [32]:
raw_train, raw_dev = get_train_test.get_train_test('../data/raw/train/audio/')

1000 : 2.8701734851995298 s
2000 : 4.307397869701845 s
3000 : 5.782134064465254 s
4000 : 7.226360297994006 s
5000 : 8.675331148853957 s


In [34]:
raw_train[0][0].shape

(99, 161)

In [35]:
x_train = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_train])
x_dev = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_dev])

y_train = np.array([x[1] for x in raw_train])
y_dev = np.array([x[1] for x in raw_dev])

In [36]:
labels_to_keep = ['yes', 'no', 'up', 'down', 'left',
                    'right', 'on', 'off', 'stop', 'go', 'silence']

for label in labels_to_keep + ['unknown']:
    print (f"{label} dev count: {len([x for x in raw_train if x[1] == label])}")

yes dev count: 180
no dev count: 179
up dev count: 178
down dev count: 174
left dev count: 175
right dev count: 181
on dev count: 174
off dev count: 176
stop dev count: 177
go dev count: 181
silence dev count: 0
unknown dev count: 3107


In [37]:
class_weight = {0: 37,
                1: 37,
                2: 37,
                3: 37,
                4: 37,
                5: 37,
                6: 37,
                7: 37,
                8: 2,
                9: 37,
                10: 37}

In [38]:
x_resh = x_train.reshape(-1, 99*161)

scaler = StandardScaler()

scaler.fit(x_resh)

x_train_scaled = scaler.transform(x_resh).reshape(-1,99,161)
x_dev_scaled = scaler.transform(x_dev.reshape(-1,99*161)).reshape(-1,99,161)

In [39]:
from tensorflow.python.keras.layers import BatchNormalization

In [ ]:
model = Sequential()
model.add(LSTM(110, return_sequences=True, input_shape=(99,161)))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(110))
model.add(BatchNormalization())
model.add(Dense(110))
model.add(BatchNormalization())
model.add(Dense(11))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

checkpointer = ModelCheckpoint(filepath='./model-{epoch:02d}.hdf5', verbose=1)

y_train_cat = to_categorical(OrdinalEncoder().fit_transform(y_train.reshape(-1,1)))
y_dev_cat = to_categorical(OrdinalEncoder().fit_transform(y_dev.reshape(-1,1)))

model.fit(x_train_scaled,y_train_cat, batch_size = 1024, epochs = 50,
                        validation_data=(x_dev_scaled,y_dev_cat), callbacks=[checkpointer], class_weight=class_weight)

Train on 4882 samples, validate on 512 samples
Epoch 1/50
4096/4882 [========================>.....] - ETA: 5s - loss: 2.8548 - categorical_accuracy: 0.1228 
Epoch 00001: saving model to ./model-01.hdf5
4882/4882 [==============================] - 42s 9ms/step - loss: 2.7953 - categorical_accuracy: 0.1274 - val_loss: 2.4000 - val_categorical_accuracy: 0.0918
Epoch 2/50
4096/4882 [========================>.....] - ETA: 4s - loss: 2.1969 - categorical_accuracy: 0.1501
Epoch 00002: saving model to ./model-02.hdf5
4882/4882 [==============================] - 27s 6ms/step - loss: 2.1825 - categorical_accuracy: 0.1477 - val_loss: 2.3898 - val_categorical_accuracy: 0.2344
Epoch 3/50
4096/4882 [========================>.....] - ETA: 3s - loss: 1.9322 - categorical_accuracy: 0.1602
Epoch 00003: saving model to ./model-03.hdf5
4882/4882 [==============================] - 26s 5ms/step - loss: 1.9155 - categorical_accuracy: 0.1630 - val_loss: 2.3676 - val_categorical_accuracy: 0.1895
Epoch 4/50
40

In [30]:
x_pred = model.predict(x_dev)

x_best_pred = np.argmax(x_pred,axis=1)

pd.DataFrame(x_best_pred.reshape(-1,1))[0].value_counts()

9    316
2    181
8      8
4      7
Name: 0, dtype: int64